!jt -t chesterish
# !jt -r

In [1]:
from genetic_algorithm import *
from numpy.random import default_rng
from tqdm import tqdm
from itertools import product #combinations_with_replacement
import binascii
%matplotlib notebook

import matplotlib.pyplot as plt
import math
from matplotlib.animation import FuncAnimation

import copy
from collections import Counter
import pandas as pd

In [2]:
## hexadecimal generator

nr_of_input = 2
nr_of_actions = 5
nr_of_inner = 3
nr_of_genes = 8
nr_individuals = 500

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## world size
ws = 40
world_size_x = ws
world_size_y = ws

##TODo if world_size_x != world_size_y fit then limit coord_full_list 

coord_full_list = np.array(list(product(range(ws), repeat=2)))
ind = np.random.choice(len(coord_full_list), nr_individuals, replace=False)
pos = coord_full_list[ind]

## initial brain and position generator

result = calculate_individual_output_weights(individuals)

## add genome

for indiv in result:
    result[indiv]['genome'] = [i.hex_id for i in individuals[indiv]]
    
## assign position and remove brains without output

indiv_to_del = []
for indiv in result:
    if result[indiv]['out']:
        result[indiv]['position'] = [list(pos[indiv])]
    else:
        indiv_to_del.append(indiv)

for key in indiv_to_del:
    del result[key] 

In [3]:
# result

In [4]:
def prevent_overlap_movement(last_pos_list, result):
    '''check if last position of each individual ovrlap with another. If yes then last posotion is switched to last but one. 
    last_pos_list - dictionary of individuals kesy and last position
    result- total info about all individuals'''
    list_of_resuls = []
    for key_1, val_1 in last_pos_list.items():
        last_pos_list_copy = copy.copy(last_pos_list)
        del last_pos_list_copy[key_1]
        list_of_keys = []
        for key_2, val_2 in last_pos_list_copy.items():
            if val_1 == val_2:
                pos2_minus = result[key_2]['position']
                pos1_minus = result[key_1]['position']
                
                if pos2_minus[-1] != pos2_minus[-2]:
                    pos2_minus[-1] = pos2_minus[-2]
                    last_pos_list[key_2] = pos2_minus[-2]
                    prevent_overlap_movement(last_pos_list, result)
                    list_of_resuls.append(2)
                elif pos2_minus[-1]== pos2_minus[-2] and pos1_minus[-1] != pos1_minus[-2]:
                    pos1_minus[-1] = pos1_minus[-2]
                    last_pos_list[key_1] = pos1_minus[-2]
                    prevent_overlap_movement(last_pos_list, result)
                    list_of_resuls.append(1)
                else:
                    list_of_resuls.append([[key_1, key_2]])
#     print(list_of_resuls)

In [5]:
nr_of_errors = 0

k = ws
n = 0
pbar = tqdm(total=k, initial=n)

while k>n: 
#     print(n)
    pos_list = [tuple(result[obj]['position'][-1]) for obj in result]
    res = list(set([ele for ele in pos_list if pos_list.count(ele) > 1]))
#     print(res)
    pbar.update(1)
    for indiv in result:
        x, y = result[indiv]['position'][-1][0], result[indiv]['position'][-1][1]
        if n<1:
            calculate_position(result, indiv, x, y, world_size_x, world_size_y)    
        elif n >= 1:
            apply_input(result, indiv)
            calculate_position(result, indiv, x, y, world_size_x, world_size_y)
    last_pos_list = {obj:result[obj]['position'][-1] for obj in result}
    prevent_overlap_movement(last_pos_list, result)
#     print()
    n += 1
pbar.close()


 35%|████████████████████████████▋                                                     | 14/40 [01:49<04:58, 11.49s/it]

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame()
for key in result:
    df[key] = pd.Series(result[key]['position']).apply(lambda x: '.'.join((str(x[0]),str(x[1]))))

print(Counter(df.iloc[-1,:]).most_common(10))

# df

generate dictionary of coords 

In [ ]:
coords = [{'x':[], 'y':[]} for key in range(len(result[indiv]['position']))]
for indiv in result:
    for pos_nr, pos in enumerate(result[indiv]['position']):
        coords[pos_nr]['x'].append(pos[0])
        coords[pos_nr]['y'].append(pos[1])

visualisation

In [ ]:
def update(i):
    ax.clear()
    ax.set_facecolor(plt.cm.Blues(.2))

    ax.set_xlim([0,world_size_x])
    ax.set_ylim([0,world_size_y])
    ax.set_title('moving')
    ax.scatter(x=coords[i]['x'],y=coords[i]['y'], c='red', s=2, marker='o')
    [spine.set_visible(False) for spine in ax.spines.values()] #remove chart 
    
fig, ax = plt.subplots(figsize=(6,6))
anime = FuncAnimation(
    fig = fig,
    func = update,
    frames = len(coords),
    interval = 10, repeat=False
)


### mutation

hexa to binary

In [ ]:
binary = bin(int(result[0]['genome'][0], 16))[2:]
if len(binary) < 32:
    factor = 32 - len(binary)
    binary = '0' * factor + binary
print(binary, result[0]['genome'][0])

print(hex(int(binary, 2))[2:])

In [ ]:
weight = 0.001
for i in range(10):
    ind = np.random.randint(0, len(binary))
    if int(binary[ind])==1:
        mutant = np.random.choice(2, 1, p=[weight, 1-weight])
        print('p0')
    else:
        mutant = np.random.choice(2, 1, p=[1-weight, weight])
        print('p1')
    
    binary_to_ints = [i for i in list(binary)]
    print(mutant)
    binary_to_ints[ind] = str(mutant[0])
    
    print(ind)
    print(''.join(binary_to_ints))  
    print(binary)
    print()

In [ ]:
str(mutant[0])

In [ ]:
binary_to_ints

In [ ]:
[int(i) for i in list(binary)]